In [1]:
import time; 

#io
import os
import json

#data
import numpy as np
import pandas as pd

#plotly
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in("justHeuristic","pyqm0lrrrw")


##Hit level

In [2]:
def collect_hits(evt):
    """collect hits (np.array) from a json event dictionary
    output: np.array([x,y,z,type_code]) of hits"""

    arrays = []
    
    #calorimeters:0
    for cal in "ECAL","HCAL":
        XYZ = [hit[1:4]+[0] for hit in evt[cal]]
        arrays.append(np.array(XYZ))
    #triggers: 1
    for trigger in "TT","OT","IT":
        XYZ1 = [hit[0:3]+[1] for hit in evt[trigger]]
        arrays.append(np.array(XYZ1))
    #muon detector (x:2,x+dx:3)
    if "MUON" in evt:
        XYZmu = np.array([hit[::2]+[2] for hit in evt["MUON"]])
        arrays.append(XYZmu)

        deltaXYZmu = np.array([hit[1::2]+[1] for hit in evt["MUON"]])
        arrays.append(XYZmu+deltaXYZmu)
    #velo detector (velor:4, velophi:5)
    for velo in "VELOR","VELOPHI":
        XYZ1 = [hit[:3]+[4] for hit in evt[velo]]
        arrays.append(np.array(XYZ1))

        XYZ2 = [hit[:3]+[5] for hit in evt[velo]]
        arrays.append(np.array(XYZ2))
    #compose a single array
    arrays = filter(lambda a:len(a)!=0,arrays)
    points = np.vstack(arrays)
    return points

In [3]:
def get_event_figure(evt):
    """builds plotly figure for an event from .json parsed dictionary"""

    particles = filter(lambda particle: particle["name"][:2] == "pi",evt["PARTICLES"])
    tracks = [particle["track"] for particle in particles]
    tracks = map(np.array,tracks)
    names = [particle["name"] for particle in particles]
    
    
    #plot track lines
    track_traces = []
    for i, (name,track) in enumerate(zip(names,tracks)):
        track_traces.append(Scatter3d(
                                name = name,
                                x=track[:,0],
                                y=track[:,1],
                                z=track[:,2],
                                mode = "lines",
                                marker = Line(
                                        #color=name
                                    )
                                ))
        
    #plot hits
    pts = collect_hits(evt)
    pts_trace = Scatter3d(
                    x=pts[:,0],
                    y=pts[:,1],
                    z=pts[:,2],
                    mode='markers',
                    marker = Marker(
                            size = '1',
                            color = pts[:,3]
                        ))
    
    #plot reconstructed Kshort decay
    ks_x,ks_y,ks_z = evt["Kdecay"]
    ks_trace = Scatter3d(
                    x=[ks_x],
                    y=[ks_y],
                    z=[ks_z],
                    mode='markers',
                    marker = Marker(
                            size = '3',
                            color = 'green'
                        ))
    
    #plot predicted Kshort dekay
    ksp_x,ksp_y,ksp_z = evt["Kdecay_predicted"]
    ksp_trace = Scatter3d(
                    x=[ksp_x],
                    y=[ksp_y],
                    z=[ksp_z],
                    mode='markers',
                    marker = Marker(
                            size = '3',
                            color='blue'
                        ))
    
    #assemble figure
    data = Data(track_traces+[pts_trace,ks_trace,ksp_trace])
    fig = Figure(data=data)
    return fig
    


#Usage example

In [6]:
json_folder="./farthest_events_bhce/"
paths = filter(lambda fname: fname.endswith(".json"),os.listdir(json_folder))
paths = np.array(map(lambda fname: os.path.join(json_folder,fname),paths))
#shuffle
paths = paths[np.argsort(np.random.random(size = len(paths)))]

#fing first non-empty event
for path in paths:
    evt = json.load(open(path))    
    if len(evt["ECAL"]) ==0:
        print "bad event"
        continue
        
    fig = get_event_figure(evt)
    break
    
time.sleep(5.)#wait for plotly to index plot
py.iplot(fig)#draw the thing!



bad event
